In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import indian
from nltk.stem.porter import PorterStemmer
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
import unicodedata
from keras.models import Sequential,Model
from keras.layers import Input,Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
# import spacy
# import contractions
from nltk.corpus import stopwords
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [72]:
df_en = pd.read_table('../data/all.tok.test.en',header=None, names=['English'])
df_th = pd.read_table('../data/all.tok.test.th', header=None, names=['Thai'])

In [55]:
def cleaning(df):
    df["English"] = df["English"].str[1:]
    df["English"] = df["English"].str.lower()

    df["English"] = df["English"].str.replace('&quot;', '')
    df["English"] = df["English"].str.replace('&apos;', '')
    df["English"] = df["English"].str.replace('&amp; lt ;', '')
    df["English"] = df["English"].str.replace('&amp; gt ;', '')
    df["English"] = df["English"].str.replace('&amp;', 'and')
    df["English"] = df["English"].str.replace('&#91;', '')
    df["English"] = df["English"].str.replace('&#93;', '')

    chars_to_remove = r'[|/\\*“”\-_~^#?.!,:;\"\'()\[\]{}<>]'
    df["English"] = df["English"].str.replace(chars_to_remove, '', regex=True)

    df["Thai"] = df["Thai"].str[1:]
    df["Thai"] = df["Thai"].str.lower()

    df["Thai"] = df["Thai"].str.replace('&amp;', 'และ')
    df["Thai"] = df["Thai"].str.replace('&', 'และ')

    chars_to_remove = r'[“”!?,\(\)\[\]\{\}<>\|\/\\:\"\'‘’\-_~^#.*]'
    df["Thai"] = df["Thai"].str.replace(chars_to_remove, '', regex=True)
    df["Thai"] = df["Thai"].str.replace('&lt ;', '')
    df["Thai"] = df["Thai"].str.replace('&gt ;', '')
    df["Thai"] = df["Thai"].str.replace('br', '')
    return df

In [63]:
from pythainlp.corpus import thai_stopwords
from nltk.corpus import stopwords

def clear_stopword( df, choice):
    sentence = []
    index = 0
    
    if choice == True:
        lang = 'Thai'
        stop_words = list(thai_stopwords())
    else:
        lang = 'English'
        stop_words = set(stopwords.words('english'))
        

    for i in df[lang]:
        filtered_sentence= []
        integrate_sentence = ""
        sentence = i.split(" ");
        for w in sentence:
            if w not in stop_words:
                filtered_sentence.append(w)
        for fw in filtered_sentence:
            if (fw != ''):
                integrate_sentence += fw + " "
        df[lang][index] = integrate_sentence
        index += 1
    
    print(sentence)
    print(filtered_sentence)
    print(integrate_sentence)
    return df

In [73]:
df_combined = pd.concat([df_en, df_th], axis=1)

In [74]:
df_combined = df_combined[df_combined['Thai'].str.contains('ประเทศไทย')]
# df_combined = df_combined[df_combined['Thai'].str.contains('สวย')]

In [75]:
df_combined = df_combined.reset_index(drop=True)
df_combined

,English,Thai
0,FThen someone mentioned Thailand . It was as s...,Fแล้ว ก็ มี คน เอ่ย ถึง ประเทศไทย ขึ้น มา ง่าย...
1,FI learned that Thailand means freedom .,Fฉัน ก็ เลย รู้ ว่า ประเทศไทย คือ อิสรภาพ
2,F- Why would you want to come here ?,F- ทำไม ลูก ถึง อยาก มา ประเทศไทย นี่ ล่ะ ?
3,FThey take drugs seriously .,Fประเทศไทย จริงจัง กับ คดียาเสพย์ ติด มาก
4,FI &apos;m still here in Thailand .,Fหนู ยัง อยู่ ที่ ประเทศไทย ค่ะ
...,...,...
67,FWe want to see it . That &apos;s why we tried...,Fแล้ว ก็ อยาก เห็น นะ ครับ เลย เลย พยายาม คิด ...
68,FIdeas for Thailand,FI deasประเทศไทย
69,FThe Asia Pacific Network of Sex Workers in Th...,FTheAsiaPacificNetworkofSexWorkers ใน ประเทศไท...
70,FThai is the official language in Thailand .,Fภาษา ไทย เป็น ภาษา ราชการ ใน ประเทศไทย


In [59]:
import pandas as pd

def read_df2csv(df):
    csv_file = "combined_data.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8')
    return True

In [77]:
read_df2csv(df_combined)

True

In [78]:
fdata = pd.read_csv('combined_data.csv')

In [79]:
fdata = cleaning(fdata)
read_df2csv(fdata)

True

In [80]:
fdata

,English,Thai
0,then someone mentioned thailand it was as sim...,แล้ว ก็ มี คน เอ่ย ถึง ประเทศไทย ขึ้น มา ง่าย ...
1,i learned that thailand means freedom,ฉัน ก็ เลย รู้ ว่า ประเทศไทย คือ อิสรภาพ
2,why would you want to come here,ทำไม ลูก ถึง อยาก มา ประเทศไทย นี่ ล่ะ
3,they take drugs seriously,ประเทศไทย จริงจัง กับ คดียาเสพย์ ติด มาก
4,i m still here in thailand,หนู ยัง อยู่ ที่ ประเทศไทย ค่ะ
...,...,...
67,we want to see it that s why we tried to come...,แล้ว ก็ อยาก เห็น นะ ครับ เลย เลย พยายาม คิด โ...
68,ideas for thailand,i deasประเทศไทย
69,the asia pacific network of sex workers in tha...,theasiapacificnetworkofsexworkers ใน ประเทศไทย...
70,thai is the official language in thailand,ภาษา ไทย เป็น ภาษา ราชการ ใน ประเทศไทย


In [87]:
def remove_long_sentences(df, column):
    df['length'] = df[column].apply(lambda x: len(x.split()))
    mask = df['length'] <= 10
    df = df[mask].drop(columns=['length'])
    return df

# ลบประโยคที่ยาวเกิน 30 คำในคอลัมน์ English และ Thai
fdata = remove_long_sentences(fdata, 'English')

In [108]:
# fdata = fdata.reset_index(drop=True)
# read_df2csv(fdata)
index = 0
for i in range(len(fdata)):
    fdata["Thai"][index] = ''.join(fdata["Thai"][index].split())
    index += 1



In [113]:
fdata["Thai"][5]

'ที่นี่ในประเทศไทยพวกคุณเชื่อว่าเพื่อนย่อมรู้ว่าเพื่อนอีกคนทำอะไร'

In [119]:
from pythainlp import word_tokenize
index = 0

for i in range(len(fdata)):
    res = ''
    tokens = word_tokenize(fdata["Thai"][index], engine='newmm')
    for i in tokens:
        res += i + ' '
    fdata["Thai"][index] = res 
    index += 1

fdata
read_df2csv(fdata)

True